In [2]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_770753/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [1]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse_sup", ascending=True, ignore_index=True)
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)

In [4]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_18_9_0,0.999962,0.687387,0.999907,0.999866,0.999894,0.000019,0.185581,0.000053,0.000079,0.000065,0.001311,0.004403,1.000004,0.004558,589.701522,987.641579,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
1,model_18_9_1,0.999962,0.687387,0.999907,0.999866,0.999894,0.000019,0.185581,0.000053,0.000079,0.000065,0.001311,0.004404,1.000004,0.004558,589.701129,987.641185,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
2,model_18_9_2,0.999962,0.687385,0.999907,0.999866,0.999894,0.000019,0.185582,0.000053,0.000080,0.000065,0.001312,0.004407,1.000004,0.004562,589.698353,987.638409,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
3,model_18_9_10,0.999962,0.687385,0.999907,0.999866,0.999894,0.000019,0.185582,0.000053,0.000080,0.000065,0.001312,0.004407,1.000004,0.004562,589.698235,987.638292,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
4,model_18_9_22,0.999962,0.687385,0.999907,0.999866,0.999894,0.000019,0.185582,0.000053,0.000080,0.000065,0.001312,0.004407,1.000004,0.004562,589.698235,987.638292,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3414,model_13_8_14,0.976621,0.499023,0.982180,0.987397,0.985686,0.011954,0.297402,0.005396,0.007391,0.006283,0.057403,0.109336,1.005650,0.113173,306.853333,515.631743,"Hidden Size=[20, 4], regularizer=0.2, learning..."
3424,model_13_8_13,0.976179,0.499641,0.981810,0.987354,0.985519,0.012181,0.297035,0.005508,0.007417,0.006356,0.057977,0.110366,1.005757,0.114240,306.815818,515.594228,"Hidden Size=[20, 4], regularizer=0.2, learning..."
3437,model_13_8_12,0.975726,0.500258,0.981431,0.987302,0.985343,0.012412,0.296669,0.005622,0.007447,0.006434,0.058556,0.111411,1.005866,0.115321,306.778131,515.556541,"Hidden Size=[20, 4], regularizer=0.2, learning..."
3446,model_13_8_11,0.975262,0.500872,0.981042,0.987240,0.985157,0.012650,0.296304,0.005740,0.007484,0.006515,0.059141,0.112470,1.005978,0.116418,306.740270,515.518680,"Hidden Size=[20, 4], regularizer=0.2, learning..."
